In [115]:
#Import
import pandas as pd
import numpy as np

#Data
import pickle

#Utilities
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, GridSearchCV, train_test_split, cross_val_score, KFold, GroupKFold
from sklearn.metrics import mean_squared_error, r2_score
import optuna

In [116]:
smarts_list = [
    '[H]',    # H atoms
    '[C,c]',  # Carbon atoms - aliphatic or aromatic
    '[O]' ,   # Oxygen atoms
    '[C;R]',  # Ring carbon
    '[c;R]',  # Aromatic carbon
    '[O;R]',  # Oxygen in ring
    '[*R]',  # Any atom in a ring
    '[OX2]', # OH and ether
    '[OX2H]', # OH
    '[c][OX2H]', # Phenolic OH
    '[OX2H0]', # Ether
    '[CX3][OX1]', # Carbonyl
    '[#6][CX3H0](=O)[#6]', #Ketone
    '[CX3H1](=O)', # Aldehyde
    '[CX3](=O)[OX2H]', # Carboxylic acid
    '[CX3](=O)O[CX3](=O)', # Anhydride
    '[CX3H0](=O)[OX2H0][#6]',  # Ester
    '[CX4H0]' ,  # Carbon sp3 with no hydrogens
    '[CX4H1]',  # Carbon sp3 with 1 hydrogen
    '[CX4H2]',  # Carbon sp3 with 2 hydrogens
    '[CX4H3]',  # Carbon sp3 with 3 hydrogens
    '[CX3]=[CX3]', # Alkenes
    '[#8].[#8]',  # Molecules containing two oxygen atoms
    '[OX2H].[OX2H]', # At least 2 OH
    '[OX2H].[OX2H].[OX2H]', # At least 3 OH
    '[CX4][OX2H]', # Typical Alcohols
    '[CX4H0][OX2H]', # Tertiary Alcohols
    '[CX4H1][OX2H]', # Secondary Alcohols
    '[CX4H2][OX2H]', # Primary Alcohols
    '[CX3H2]=[CX3H1]',  # CH2=CH-
    '[CX3H2]=[CX3H0]',  # CH2=C()-
    '[CX3H1]=[CX3H1]',  # -CH=CH-
    '[OX2H0].[OX2H0]', # At least 2 ethers
    '[OX2H0].[OX2H0].[OX2H0]', # At least 3 ethers 
    '[c][OX2][c]', # Aromatic-Aromatic ether
    '[c][OX2][C]' , # Aromatic-Aliphatic ether
    '[#8][#6][#6][#8]', #1,2-diol or diether
    '[#8][#6][#6][#6][#8]', #1,3-diol or diether
    '[#6;R]-O-[#6;R]', #Cyclic ether
]

smarts_labels = [f'X{i+1}' for i in range(len(smarts_list))]

In [117]:
df = pd.read_excel("DIPPR.xlsx", sheet_name=0)
df.drop(columns=['Name', 'CAS', 'Family'], inplace=True)

# Group by SMILE and T, take mean of K
df_reduced = df.groupby(['SMILE', 'T'], as_index=False)['K'].mean()

In [118]:
#Polynpomial interpolation
interpolated_dfs = []

for smile, group in df_reduced.groupby('SMILE'):
    group = group.sort_values('T')

    # Reduce to at most 10 diverse points (evenly spread)
    if len(group) > 10:
        indices = np.linspace(0, len(group) - 1, 10, dtype=int)
        group = group.iloc[indices].reset_index(drop=True)

    # Interpolate over a denser T range if desired
    T_new = np.linspace(group['T'].min(), group['T'].max(), num=10)

    if len(group) >= 4: # Change if needed
        # Safe to fit cubic
        coeffs = np.polyfit(group['T'], group['K'], deg=2)
        poly = np.poly1d(coeffs)
        K_new = poly(T_new)
    
    else:
        # Fall back to linear if not enough points
        coeffs = np.polyfit(group['T'], group['K'], deg=1)
        poly = np.poly1d(coeffs)
        K_new = poly(T_new)


    interpolated = pd.DataFrame({
        'SMILE': smile,
        'T': T_new,
        'K': K_new
    })
    interpolated_dfs.append(interpolated)

result = pd.concat(interpolated_dfs, ignore_index=True)

In [119]:
# Convert SMILES to Mol objects
result['mol'] = result['SMILE'].apply(Chem.MolFromSmiles)

# Add explicit hydrogens to all molecules
result['mol'] = result['mol'].apply(lambda mol: Chem.AddHs(mol) if mol else None)

# Generate binary SMARTS descriptors
for smarts, label in zip(smarts_list, smarts_labels):
    pattern = Chem.MolFromSmarts(smarts)
    result[label] = result['mol'].apply(lambda mol: len(mol.GetSubstructMatches(pattern)) if mol else 0)

# Calculate molecular weight and add it to the DataFrame
result['MolWeight'] = result['mol'].apply(lambda mol: Descriptors.MolWt(mol) if mol else None)

In [120]:
X = result.drop(columns=['SMILE', 'mol', 'K'])
y = result['K']

In [121]:
# Use SMILE as the group identifier for splitting
groups = result['SMILE']
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [122]:
cv = GroupKFold(n_splits=5)
groups_train = groups.iloc[train_idx]
scores = []

def objectives(trial):
    # Suggest hyperparameters
    C = trial.suggest_float('C', 10, 300, log=True)
    epsilon = trial.suggest_float('epsilon', 4e-3, 1e-2, log=True)
    gamma = trial.suggest_float('gamma', 2e-4, 1e-3, log=True)

    model = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2', groups=groups_train, n_jobs=-1)
    return np.mean(scores)

study = optuna.create_study(direction='maximize')
study.optimize(objectives, n_trials=200)

[I 2025-07-01 16:03:22,772] A new study created in memory with name: no-name-1f84eba5-deb2-450b-882b-691eb8d94db4
[I 2025-07-01 16:03:36,595] Trial 0 finished with value: 0.6825867474771795 and parameters: {'C': 122.64115701662915, 'epsilon': 0.004463489429846331, 'gamma': 0.0002101241095733099}. Best is trial 0 with value: 0.6825867474771795.
[I 2025-07-01 16:03:39,830] Trial 1 finished with value: 0.6790214934328509 and parameters: {'C': 135.31595497100622, 'epsilon': 0.007688435938910573, 'gamma': 0.0005178675683097455}. Best is trial 0 with value: 0.6825867474771795.
[I 2025-07-01 16:03:42,473] Trial 2 finished with value: 0.6828039223270191 and parameters: {'C': 215.5507116449624, 'epsilon': 0.007252518747555857, 'gamma': 0.0005194421147483017}. Best is trial 2 with value: 0.6828039223270191.
[I 2025-07-01 16:03:42,679] Trial 3 finished with value: 0.6782551039046918 and parameters: {'C': 78.57418876096601, 'epsilon': 0.00804931618511133, 'gamma': 0.000479189157882991}. Best is tr

In [123]:
optuna.visualization.plot_optimization_history(study).show()

In [124]:
best_params = study.best_params
print("Best parameters from Optuna:", best_params)

Best parameters from Optuna: {'C': 25.083047520012443, 'epsilon': 0.007113687959490875, 'gamma': 0.00021443823870953256}


In [125]:
best_params = {'C': 45.515217222968296, 'epsilon': 0.004022852222818151, 'gamma': 0.00020233531522643092}

In [126]:
final_model = SVR(**best_params)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
print(f"Final Test R²: {test_r2:.3f}")

Final Test R²: 0.949


SAVE BEST MODEL

In [127]:
best_params = {'C': 45.515217222968296, 'epsilon': 0.004022852222818151, 'gamma': 0.00020233531522643092}
final_model = SVR(**best_params)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
test_r2 = r2_score(y_test, y_pred)
print(f"Final Test R²: {test_r2:.3f}")

Final Test R²: 0.949


In [128]:
# Save model to pkl
with open('svr_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

APPLY MODEL TO SDB

In [129]:
sdb = pd.read_excel("sdb1_unique_pures+predict.xlsx", sheet_name=0)

# Convert SMILES to Mol objects
sdb['mol'] = sdb['SMILES'].apply(Chem.MolFromSmiles)

# Add explicit hydrogens to all molecules
sdb['mol'] = sdb['mol'].apply(lambda mol: Chem.AddHs(mol) if mol else None)

# Generate binary SMARTS descriptors
for smarts, label in zip(smarts_list, smarts_labels):
    pattern = Chem.MolFromSmarts(smarts)
    sdb[label] = sdb['mol'].apply(lambda mol: len(mol.GetSubstructMatches(pattern)) if mol else 0)

# Calculate molecular weight and add it to the DataFrame
sdb['MolWeight'] = sdb['mol'].apply(lambda mol: Descriptors.MolWt(mol) if mol else None)

X = sdb.drop(columns=['Name', 'SMILES', 'mol', 'K', 'K_pred', '% difference'])
y = sdb['K']

In [130]:
final_model = pickle.load(open('svr_model.pkl', 'rb'))

In [131]:
preds_svr = final_model.predict(X)
r2_svr = r2_score(sdb['K'], preds_svr)
r2_moreno= r2_score(sdb['K'], sdb['K_pred'])
print(f"R² for xgb predictions: {r2_svr:.3f}")
print(f"R² for Moreno's predictions: {r2_moreno:.3f}")

R² for xgb predictions: 0.962
R² for Moreno's predictions: 0.949


In [133]:
# Create df with columns y_test, y_pred, sdb['K'], preds_xgb
results_df = pd.DataFrame({
    'y_test': y_test,
    'SVR_pred': y_pred,
})

results_df2 = pd.DataFrame({
        'sdb_K': sdb['K'],
    'SVR_pred_SDB1': preds_svr
})

# Save the results to an Excel file
results_df.to_excel('placeholder.xlsx', index=False)
results_df2.to_excel('placeholder2.xlsx', index=False)

APPLY TO ALL DB (Replace Exp)

In [104]:
db = pd.read_excel('../report/DB.xlsx', sheet_name=1)

In [106]:
df_A = db[['SMILES_A', 'T']].copy()
df_A.rename(columns={'SMILES_A': 'SMILES'}, inplace=True)
df_B = db[['SMILES_B', 'T']].copy()
df_B.rename(columns={'SMILES_B': 'SMILES'}, inplace=True)

In [108]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd

def generate_smarts_descriptors(df, smiles_column='SMILES'):
    """
    Adds molecular representations, SMARTS-based substructure counts,
    and molecular weight to a DataFrame.

    Parameters:
    - df: pandas DataFrame with a column of SMILES strings.
    - smiles_column: name of the column containing SMILES (default: 'SMILE').

    Returns:
    - Modified DataFrame with SMARTS descriptors and MolWeight column.
    """
    
    smarts_list = [
        '[H]', '[C,c]', '[O]', '[C;R]', '[c;R]', '[O;R]', '[*R]', '[OX2]', '[OX2H]',
        '[c][OX2H]', '[OX2H0]', '[CX3][OX1]', '[#6][CX3H0](=O)[#6]', '[CX3H1](=O)',
        '[CX3](=O)[OX2H]', '[CX3](=O)O[CX3](=O)', '[CX3H0](=O)[OX2H0][#6]', '[CX4H0]',
        '[CX4H1]', '[CX4H2]', '[CX4H3]', '[CX3]=[CX3]', '[#8].[#8]', '[OX2H].[OX2H]',
        '[OX2H].[OX2H].[OX2H]', '[CX4][OX2H]', '[CX4H0][OX2H]', '[CX4H1][OX2H]',
        '[CX4H2][OX2H]', '[CX3H2]=[CX3H1]', '[CX3H2]=[CX3H0]', '[CX3H1]=[CX3H1]',
        '[OX2H0].[OX2H0]', '[OX2H0].[OX2H0].[OX2H0]', '[c][OX2][c]', '[c][OX2][C]',
        '[#8][#6][#6][#8]', '[#8][#6][#6][#6][#8]', '[#6;R]-O-[#6;R]'
    ]
    smarts_labels = [f'X{i+1}' for i in range(len(smarts_list))]

    df = df.copy()
    
    # Generate RDKit Mol objects and add explicit Hs
    df['mol'] = df[smiles_column].apply(Chem.MolFromSmiles)
    df['mol'] = df['mol'].apply(lambda mol: Chem.AddHs(mol) if mol else None)

    # Calculate SMARTS-based substructure counts
    for smarts, label in zip(smarts_list, smarts_labels):
        pattern = Chem.MolFromSmarts(smarts)
        df[label] = df['mol'].apply(lambda mol: len(mol.GetSubstructMatches(pattern)) if mol else 0)

    # Add molecular weight
    df['MolWeight'] = df['mol'].apply(lambda mol: Descriptors.MolWt(mol) if mol else None)

    df.drop(columns=['mol'], inplace=True)  # Optionally drop the Mol column
    return df

In [109]:
df_A = generate_smarts_descriptors(df_A, smiles_column='SMILES')
df_B = generate_smarts_descriptors(df_B, smiles_column='SMILES')

In [110]:
df_A

,SMILES,T,X1,X2,X3,X4,X5,X6,X7,X8,...,X31,X32,X33,X34,X35,X36,X37,X38,X39,MolWeight
0,CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O,304.01,20,10,1,6,0,0,6,1,...,0,0,0,0,0,0,0,0,0,156.269
1,CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O,314.00,20,10,1,6,0,0,6,1,...,0,0,0,0,0,0,0,0,0,156.269
2,CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O,324.01,20,10,1,6,0,0,6,1,...,0,0,0,0,0,0,0,0,0,156.269
3,CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O,333.98,20,10,1,6,0,0,6,1,...,0,0,0,0,0,0,0,0,0,156.269
4,CC(C)[C@@H]1CC[C@@H](C)C[C@H]1O,344.15,20,10,1,6,0,0,6,1,...,0,0,0,0,0,0,0,0,0,156.269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048,CCCCCCCCCCC,322.02,24,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,156.313
3049,CCCCCCCCCCC,331.75,24,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,156.313
3050,CCCCCCCCCCC,341.92,24,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,156.313
3051,CCCCCCCCCCC,351.90,24,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,156.313


In [149]:
def find_identical_columns(df):
    identical_columns = []
    columns = df.columns
    for i in range(len(columns)):
        for j in range(i + 1, len(columns)):
            if df[columns[i]].equals(df[columns[j]]):
                identical_columns.append((columns[i], columns[j]))
    return identical_columns

# Example usage
matches = find_identical_columns(df_A)
print(matches)

[('X6', 'X10'), ('X6', 'X12'), ('X6', 'X14'), ('X6', 'X27'), ('X6', 'X30'), ('X6', 'X36'), ('X10', 'X12'), ('X10', 'X14'), ('X10', 'X27'), ('X10', 'X30'), ('X10', 'X36'), ('X12', 'X14'), ('X12', 'X27'), ('X12', 'X30'), ('X12', 'X36'), ('X14', 'X27'), ('X14', 'X30'), ('X14', 'X36'), ('X27', 'X30'), ('X27', 'X36'), ('X30', 'X36'), ('X35', 'X39')]


In [111]:
df_B

,SMILES,T,X1,X2,X3,X4,X5,X6,X7,X8,...,X31,X32,X33,X34,X35,X36,X37,X38,X39,MolWeight
0,CCCCCCCCCC(O)=O,304.01,20,10,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,172.268
1,CCCCCCCCCC(O)=O,314.00,20,10,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,172.268
2,CCCCCCCCCC(O)=O,324.01,20,10,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,172.268
3,CCCCCCCCCC(O)=O,333.98,20,10,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,172.268
4,CCCCCCCCCC(O)=O,344.15,20,10,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,172.268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048,CCCCCCCCCCCCCC(=O)OC,322.02,30,15,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,242.403
3049,CCCCCCCCCCCCCC(=O)OC,331.75,30,15,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,242.403
3050,CCCCCCCCCCCCCC(=O)OC,341.92,30,15,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,242.403
3051,CCCCCCCCCCCCCC(=O)OC,351.90,30,15,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,242.403


In [150]:
matches_B = find_identical_columns(df_B)
print(matches_B)

[('X6', 'X39'), ('X10', 'X12'), ('X10', 'X16'), ('X10', 'X31'), ('X10', 'X32'), ('X10', 'X35'), ('X10', 'X36'), ('X12', 'X16'), ('X12', 'X31'), ('X12', 'X32'), ('X12', 'X35'), ('X12', 'X36'), ('X16', 'X31'), ('X16', 'X32'), ('X16', 'X35'), ('X16', 'X36'), ('X22', 'X30'), ('X31', 'X32'), ('X31', 'X35'), ('X31', 'X36'), ('X32', 'X35'), ('X32', 'X36'), ('X35', 'X36')]


In [151]:
# Find overlap of match and matches_B
overlap = set(matches) & set(matches_B)
print("Overlap between matches and matches_B:", overlap)

Overlap between matches and matches_B: {('X10', 'X12'), ('X10', 'X36'), ('X12', 'X36')}


In [65]:
df_A['K'] = final_model.predict(df_A.drop(columns=['SMILES']))
df_B['K'] = final_model.predict(df_B.drop(columns=['SMILES']))

db = pd.read_excel('../report/DB.xlsx')
db.drop(['KA', 'KB'], axis=1, inplace=True)
db['KA'] = df_A['K']
db['KB'] = df_B['K']
db.to_excel('../report/DB_svr_predict_all.xlsx', index=False)

APPLY TO UNIQUE DB

In [66]:
final_model.get_params

<bound method BaseEstimator.get_params of SVR(C=45.515217222968296, epsilon=0.004022852222818151,
    gamma=0.00020233531522643092)>

In [67]:
db_unique = pd.read_excel('DB_unique.xlsx')
db_unique = generate_smarts_descriptors(db_unique, smiles_column='SMILES')
db_unique['K'] = final_model.predict(db_unique.drop(columns=['SMILES', 'Name']))

In [68]:
db_unique_saved = db_unique[['Name', 'SMILES', 'T', 'K']].copy()
db_unique_saved.to_excel('DB_unique_svr_predictions.xlsx', index=False)

CALCULATE ADDITIONAL ANALYSIS

In [134]:
df_analyze = pd.read_excel('../report/results.xlsx', sheet_name=1)
df_analyze

,test_set,XGB_pred,SVR_pred,Moreno_pred,K_SDB1,XGB_pred_SDB1,SVR_pred_SDB1,Moreno_pred_SDB1
0,0.163539,0.161307,0.173020,0.157133,0.1538,0.150287,0.155675,0.170709
1,0.159778,0.160371,0.172414,0.156413,0.1473,0.143931,0.152295,0.164745
2,0.156518,0.161180,0.171670,0.155691,0.1257,0.136805,0.133990,0.129691
3,0.153759,0.160659,0.170807,0.154972,0.1249,0.128831,0.125759,0.122389
4,0.151500,0.161839,0.169844,0.154253,0.1355,0.141646,0.141275,0.137057
...,...,...,...,...,...,...,...,...
345,0.147769,0.135470,0.131603,0.143565,NaN,NaN,NaN,NaN
346,0.145699,0.133414,0.129944,0.141259,NaN,NaN,NaN,NaN
347,0.143544,0.131122,0.128526,0.139010,NaN,NaN,NaN,NaN
348,0.141305,0.130356,0.128059,0.136823,NaN,NaN,NaN,NaN


In [147]:
xgb_df = df_analyze[['test_set', 'XGB_pred']].dropna()
rmse_xgb = np.sqrt(mean_squared_error(xgb_df['test_set'], xgb_df['XGB_pred']))

svr_df = df_analyze[['test_set', 'SVR_pred']].dropna()
rmse_svr = np.sqrt(mean_squared_error(svr_df['test_set'], svr_df['SVR_pred']))

moreno_df = df_analyze[['test_set', 'Moreno_pred']].dropna()
rmse_moreno = np.sqrt(mean_squared_error(moreno_df['test_set'], moreno_df['Moreno_pred']))

print(f"RMSE for XGB predictions: {rmse_xgb}")
print(f"RMSE for SVR predictions: {rmse_svr}")
print(f"RMSE for Moreno's predictions: {rmse_moreno}")


RMSE for XGB predictions: 0.00566648183145309
RMSE for SVR predictions: 0.007012624376741899
RMSE for Moreno's predictions: 0.013869422634938724


In [148]:
mae_xgb = np.mean(np.abs(xgb_df['test_set'] - xgb_df['XGB_pred']))
mae_svr = np.mean(np.abs(svr_df['test_set'] - svr_df['SVR_pred']))
mae_moreno = np.mean(np.abs(moreno_df['test_set'] - moreno_df['Moreno_pred']))

print(f"MAE for XGB predictions: {mae_xgb}")
print(f"MAE for SVR predictions: {mae_svr}")
print(f"MAE for Moreno's predictions: {mae_moreno}")


MAE for XGB predictions: 0.003978275696510666
MAE for SVR predictions: 0.005034940701970073
MAE for Moreno's predictions: 0.006901533926826764
